# 03 • Double Well potential

Consider the Hamiltonian of a 1D particle in a double well potential:

$$H(q, p) = T(p) + V(q) = \frac{p^2}{2m} + (q-r_0)(q-r_1)(q-r_2)(q-r_3)$$

- $T(p)$ and $V(q)$ are the kinetic and potential energies,
- $q$ and $p$ are the generalized coordinate and momentum,
- $m$ is the mass and $r_0,\dots,r_3$ are the roots of polynomial describing the double well.

In this notebook, we are simulating the above system using a symplectic Euler integrator

$$\begin{align}
p(t+\Delta t) &= p(t) + \Delta t \cdot \frac{\text{d}p}{\text{d}t}(q(t), p(t)) \quad & \text{(kick)} \\
q(t+\Delta t) &= q(t) + \Delta t \cdot \frac{\text{d}q}{\text{d}t}(q(t), p(t+\Delta t)) & \quad \text{(drift)}
\end{align}$$

This is the same update as in `Hamiltonian.step()`, but the integrator allows to make multiple steps in one call and to apply a Langevin thermostat

$$p_{\text{therm}}(t+\Delta t) = p(t+\Delta t) \cdot e^{-\gamma\Delta t} + \sqrt{m \text{k}_{\text{B}}T(1 - e^{-2\gamma\Delta t})} \cdot \mathcal{N}(0, 1) \quad \text{(Ornstein-Uhlenbeck)}$$

with damping $\gamma$, mass(es) $m$, thermal energy $\text{k}_{\text{B}}T$, and normally distributed random numbers $\mathcal{N}(0, 1)$ (which have to match $p$ in shape).

In [ ]:
import matplotlib.pyplot as plt
import torch

from hamiltonian import DoubleWell
from integrator import SymplecticEuler

H = DoubleWell()

q = torch.linspace(-2.3, 2.3, 100)
V = H.epot(q)

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(q, V)
ax.set_xlabel("q / a.u.")
ax.set_ylabel("V / a.u.")
fig.tight_layout()

With default parameters, the double well shows a the global minimum $-2<q_{\text{global}}<-1$ and a local minimum $1<q_{\text{local}}<2$. Both minima are separated by a potential energy barrier $0<q_{\text{barrier}}<1$.

# Integration without thermostat

Let's start three separate simulations without thermostat (i.e., microcanconical ensemble) with particles at rest ($p(0)=0$):

- left of the barrier, $q(0) = 0$
- right of the barrier, $q(0) = 1$
- right of the local minimum and at a potential higher than the barrier, $q(0) = 2.2$

Each trajectory runs for 1000 steps with $\Delta t=0.015$:

In [ ]:
delta_t = 0.015
q = torch.tensor([0.0, 1.0, 2.2])
p = torch.tensor([0.0, 0.0, 0.0])

integrate = SymplecticEuler(H, delta_t)

t, qt, pt = integrate(q, p, 1000)

qt_0, qt_1, qt_2 = qt.T
pt_0, pt_1, pt_2 = pt.T

fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

axes[0].plot(t, qt_0, label="$q(0)$=0")
axes[0].plot(t, qt_1, label="$q(0)$=1")
axes[0].plot(t, qt_2, label="$q(0)$=2.2")
axes[0].set_ylabel("q / a.u.")

axes[1].plot(t, pt_0, label="$q(0)$=0")
axes[1].plot(t, pt_1, label="$q(0)$=1")
axes[1].plot(t, pt_2, label="$q(0)$=2.2")
axes[1].set_ylabel("p / a.u.")

for ax in axes.flat:
    ax.set_xlabel("t / a.u.")
    ax.legend()
fig.tight_layout()

In the first two cases, the trajectories stay within their respective potential valleys as their energies are insufficient to cross the barrier. Total energy is (approximately) conserved and the system is limited by its initial conditions. In the last case, the system started with enough potential energy to move freely between both valleys.

## Integration with damping

Let's look at a single starting point just left of the barrier ($q(0)=0$) with particle at rest ($p(0)=0$). We now choose a damping parameter $\gamma=2$ and integrate for 1000 steps with $\Delta t=0.015$:

In [ ]:
q = torch.tensor([0.0])
p = torch.tensor([0.0])

integrate = SymplecticEuler(H, delta_t, damping=2)

t, qt, pt = integrate(q, p, 1000)
t, qt, pt = t.squeeze(), qt.squeeze(), pt.squeeze()

fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

axes[0].plot(t, qt, label="q")
axes[0].plot(t, pt, label="p")
axes[0].set_ylabel("q, p / a.u.")

axes[1].plot(t, H.ekin(pt), label="T")
axes[1].plot(t, H.epot(qt), label="V")
axes[1].plot(t, H(qt, pt), label="H")
axes[1].set_ylabel("energy / a.u.")

for ax in axes.flat:
    ax.set_xlabel("t / a.u.")
    ax.legend()
fig.tight_layout()

The particle makes a few oscillations in the expected valley before slowly coming to a halt as the decay factor $e^{-\gamma \Delta t}$ transfers kinetit energy out of the system.

## Integration with thermostat

We repeat the previous exercise (start from $q(0)=0$, $p(0)=0$, $\gamma=2$), but add thermal energy $\text{k}_{\text{B}}T=5$.

In [ ]:
q = torch.tensor([0.0])
p = torch.tensor([0.0])

integrate = SymplecticEuler(H, delta_t, damping=2, thermal_energy=5)

t, qt, pt = integrate(q, p, 1000)
qt, pt = qt.squeeze(), pt.squeeze()

fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

axes[0].plot(t, qt, label="q")
axes[0].plot(t, pt, label="p")
axes[0].set_ylabel("q, p / a.u.")

axes[1].plot(t, H.ekin(pt), label="T")
axes[1].plot(t, H.epot(qt), label="V")
axes[1].plot(t, H(qt, pt), label="H")
axes[1].set_ylabel("energy / a.u.")

for ax in axes.flat:
    ax.set_xlabel("t / a.u.")
    ax.legend()
fig.tight_layout()

This time, the system does not come to a halt or remains in its initial valley. Its *contact to a heat bath* (i.e., canonical ensemble) allows the particle to occasionally cross the barrier despite starting out with less energy than needs to do so.

To summarize: we have added another Hamiltonian for a 1D particle in a double well potential and a symplectic Euler integrator to simulate the time evolution of arbitrary Hamiltonians.

In the next notebook, we are looking into deep operator networks.